In [2]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
from mpl_toolkits.basemap import Basemap, shiftgrid
from netCDF4 import Dataset
%matplotlib inline

In [3]:
# The GFS output are avaiable in directories that are named by the forecast
#  date/time.  For example, the directory for the forecast for August 28, 2015
#  would be 20150828.  There are a few forecasts run each day, but here we'll
#  try pull from the 12:00 run.

# Here we try to get "today"
#forecast_date = '20200411'
forecast_date = datetime.datetime.now().strftime('%Y%m%d')
url = 'http://nomads.ncep.noaa.gov:80/dods/gfs_0p50/gfs' + forecast_date + '/gfs_0p50_12z'
data = Dataset(url)

In [4]:
# NOTE: for whatever reason, this section takes 10-15 minutes to complete; no idea why
# read lats,lons
# reverse latitudes so they go from south to north.
latitudes = data.variables['lat'][::-1]
longitudes = data.variables['lon'][:].tolist()

# get sea level pressure and 10-m wind data.
# mult slp by 0.01 to put in units of hPa.
slpin = 0.01*data.variables['pressfc'][:].squeeze()
uin = data.variables['ugrd10m'][:].squeeze()
vin = data.variables['vgrd10m'][:].squeeze()

In [5]:
# Again, let's just plot the last (time) value
slp = slpin[-1,:,:]
u = uin[-1,:,:]
v = vin[-1,:,:]

In [ ]:
# make 2-d grid of lons, lats
lons, lats = np.meshgrid(longitudes,latitudes)

# make orthographic basemap.
m = Basemap(resolution='c',projection='ortho',lat_0=60.,lon_0=-60.)

# create figure, add axes
fig1 = plt.figure(figsize=(8,10))
ax = fig1.add_axes([0.1,0.1,0.8,0.8])

# set desired contour levels.
clevs = np.arange(960,1061,5)

# compute native x,y coordinates of grid.
x, y = m(lons, lats)

# define parallels and meridians to draw.
parallels = np.arange(-80.,90,20.)
meridians = np.arange(0.,360.,20.)

# plot SLP contours.
CS1 = m.contour(x,y,slp,clevs,linewidths=0.5,colors='k',animated=True)
CS2 = m.contourf(x,y,slp,clevs,cmap=plt.cm.RdBu_r,animated=True)

In [1]:
# make 2-d grid of lons, lats
lons, lats = np.meshgrid(longitudes,latitudes)

# make orthographic basemap.
m = Basemap(resolution='c',projection='ortho',lat_0=60.,lon_0=-60.)

# create figure, add axes
fig1 = plt.figure(figsize=(8,10))
ax = fig1.add_axes([0.1,0.1,0.8,0.8])

# set desired contour levels.
clevs = np.arange(960,1061,5)

# compute native x,y coordinates of grid.
x, y = m(lons, lats)

# define parallels and meridians to draw.
parallels = np.arange(-80.,90,20.)
meridians = np.arange(0.,360.,20.)

# plot SLP contours.
CS1 = m.contour(x,y,slp,clevs,linewidths=0.5,colors='k',animated=True)
CS2 = m.contourf(x,y,slp,clevs,cmap=plt.cm.RdBu_r,animated=True)

# plot wind vectors on projection grid.
# first, shift grid so it goes from -180 to 180 (instead of 0 to 360
# in longitude).  Otherwise, interpolation is messed up.
ugrid,newlons = shiftgrid(180.,u,longitudes,start=False)
vgrid,newlons = shiftgrid(180.,v,longitudes,start=False)

# transform vectors to projection grid.
uproj,vproj,xx,yy = \
m.transform_vector(ugrid,vgrid,newlons,latitudes,31,31,returnxy=True,masked=True)

# now plot.
Q = m.quiver(xx,yy,uproj,vproj,scale=700)

# make quiver key.
qk = plt.quiverkey(Q, 0.1, 0.1, 20, '20 m/s', labelpos='W')

# draw coastlines, parallels, meridians.
m.drawcoastlines(linewidth=1.5)
m.drawparallels(parallels)
m.drawmeridians(meridians)

# add colorbar
cb = m.colorbar(CS2,"bottom", size="5%", pad="2%")
cb.set_label('hPa')

# set plot title
ax.set_title('SLP and Wind Vectors '+str(date))
plt.show()

# create 2nd figure, add axes
fig2 = plt.figure(figsize=(8,10))
ax = fig2.add_axes([0.1,0.1,0.8,0.8])

# plot SLP contours
CS1 = m.contour(x,y,slp,clevs,linewidths=0.5,colors='k',animated=True)
CS2 = m.contourf(x,y,slp,clevs,cmap=plt.cm.RdBu_r,animated=True)

# plot wind barbs over map.
barbs = m.barbs(xx,yy,uproj,vproj,length=5,barbcolor='k',flagcolor='r',linewidth=0.5)

# draw coastlines, parallels, meridians.
m.drawcoastlines(linewidth=1.5)
m.drawparallels(parallels)
m.drawmeridians(meridians)

# add colorbar
cb = m.colorbar(CS2,"bottom", size="5%", pad="2%")
cb.set_label('hPa')

# set plot title.
ax.set_title('SLP and Wind Barbs '+str(date))

NameError: name 'np' is not defined

In [7]:
print(data.variables)

OrderedDict([('time', <class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    grads_dim: t
    grads_mapping: linear
    grads_size: 129
    grads_min: 12z11apr2020
    grads_step: 3hr
    units: days since 1-1-1 00:00:0.0
    long_name: time
    minimum: 12z11apr2020
    maximum: 12z27apr2020
    resolution: 0.125
unlimited dimensions: 
current shape = (129,)
filling off
), ('lev', <class 'netCDF4._netCDF4.Variable'>
float64 lev(lev)
    grads_dim: z
    grads_mapping: levels
    units: millibar
    long_name: altitude
    minimum: 1000.0
    maximum: 0.4
    resolution: 20.4
unlimited dimensions: 
current shape = (50,)
filling off
), ('lat', <class 'netCDF4._netCDF4.Variable'>
float64 lat(lat)
    grads_dim: y
    grads_mapping: linear
    grads_size: 361
    units: degrees_north
    long_name: latitude
    minimum: -90.0
    maximum: 90.0
    resolution: 0.5
unlimited dimensions: 
current shape = (361,)
filling off
), ('lon', <class 'netCDF4._netCDF4.Variable'>
float64 lon(lon)
 